Trying to make a fake dataset here to be able to test how featurising these bravais lattices will work with PHF features

In [10]:
import pandas as pd
import numpy as np
import structures as st
import plotly.graph_objects as go

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy
from gtda.diagrams import NumberOfPoints
from gtda.diagrams import Amplitude

from sklearn.pipeline import make_union, Pipeline

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [13]:
def persistence_diagrams(coords):
    # Gives data to make persitenace diagrams, need to make code for that
    # Track connected components, loops, and voids
    homology_dimensions = [0, 1, 2]

    # Collapse edges to speed up H2 persistence calculation!
    persistence = VietorisRipsPersistence(
        metric="euclidean",
        homology_dimensions=homology_dimensions,
        n_jobs=1,
        collapse_edges=True,
    )
    
    reshaped_coords=coords[None, :, :]
    diagrams_basic = persistence.fit_transform(reshaped_coords)
    return coords, diagrams_basic

def make_pipeline():

    metrics = [
        {"metric": metric}
        for metric in ["bottleneck", "wasserstein", "landscape", "persistence_image"]
    ]

    # Concatenate to generate 3 + 3 + (4 x 3) = 18 topological features
    feature_union = make_union(
        PersistenceEntropy(normalize=True),
        NumberOfPoints(n_jobs=1),
        *[Amplitude(**metric, n_jobs=1) for metric in metrics]
    )

    ## then we use a pipeline to transform, the data and spit i out
    # mwah hahahahaha
    pipe = Pipeline(
        [
            ("features", feature_union)
        ]
    )
        
    return pipe

def featurising_coords(coords_of_structures):
    topol_feat_list = []
    pipe = make_pipeline()

    for coords in coords_of_structures:
        _ , diagrams_basic = persistence_diagrams(coords)
        X_basic = pipe.fit_transform(diagrams_basic)
        # topology feat list stores the topological features for each structure
        topol_feat_list.append([x for x in X_basic[0]])
    
    # topol feat mat is a matrix of topological features
    topol_feat_mat = np.array(topol_feat_list)
    
    return topol_feat_mat, topol_feat_list

def do_random_forest(features, target):
    # we need to binarize the labels
    mlb = MultiLabelBinarizer()
    features_transformed = mlb.fit_transform(features)

    x_train, x_test ,y_train, y_test = train_test_split(features_transformed,target,test_size=0.2, random_state=42)

    model = RandomForestClassifier(n_estimators=100, random_state=42)

    model.fit(x_train, y_train)

    accuracy = round(model.score(x_test, y_test) * 100, 2)
    print(f"Accuracy of the random forest: {accuracy}%")
    
    y_pred = model.predict(x_test)
    
    f1 = f1_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')

    print(f"F1 Score: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    
    return f1, precision, recall, accuracy

In [14]:
def randomise_sides(length, num_of_diff_sides):
    sides_df = pd.DataFrame(columns=['a', 'b', 'c'])
    
    while len(sides_df) < length:
        side_a = round(np.random.uniform(3, 9.0), 3)
        if num_of_diff_sides == 1:
            side_b = side_a
            side_c  = round(np.random.uniform(3, 9.0), 3)
            
            if side_a != side_c:
                sides_df = sides_df.append({'a': side_a, 'b': side_b, 'c': side_c}, ignore_index=True)
            
        elif num_of_diff_sides == 2:
            side_b = round(np.random.uniform(3, 9.0), 3)
            side_c  = round(np.random.uniform(3, 9.0), 3)
            
            if side_a != side_b != side_c:
                sides_df = sides_df.append({'a': side_a, 'b': side_b, 'c': side_c}, ignore_index=True)
                
        else:
            side_b = side_a
            side_c = side_a
            sides_df = sides_df.append({'a': side_a, 'b': side_b, 'c': side_c}, ignore_index=True)
            
    return sides_df

In [15]:
def randomise_angles(length, randomise_1, randomise_3, same_3_randomised):
    if randomise_1 == True:
        angles = []
        while len(angles) < length:
            ang_a = np.random.uniform(45, 80)
            ang_a = round(ang_a, 1)
            angles.append(ang_a)
            
    elif randomise_3 == True:
        angles = []
        while len(angles) < length:
            ang_a = np.random.uniform(45, 80)
            ang_a = round(ang_a, 1)
            ang_b = np.random.uniform(45, 80)
            ang_b = round(ang_b, 1)
            ang_y = np.random.uniform(45, 80)
            ang_y = round(ang_y, 1)
            
            if ang_a != ang_b != ang_y:
                angles.append([ang_a, ang_b, ang_y])
                
    elif same_3_randomised == True:
        angles = []
        while len(angles) < length:
            ang_a = np.random.uniform(45, 80)
            ang_a = round(ang_a, 1)
            ang_b = ang_a
            ang_y = ang_a
            
            angles.append([ang_a, ang_b, ang_y])
            
    else:
        print('Please choose a valid option')
        
    return angles

# try_ang = randomise_angles(10, False, True, False)
# try_ang_df = pd.DataFrame(try_ang, columns=['angle_a', 'angle_b', 'angle_y'])

In [16]:
def make_dataset(a, b, c, alpha, beta, gamma, name):
    data = {
        'a': a,
        'b': b,
        'c': c,
        'alpha': alpha,
        'beta': beta,
        'gamma': gamma,
        'structure type': name
        }
    
    df = pd.DataFrame(data)
    return df

In [9]:
lengths = [100, 200, 400, 600, 800, 1000]
unit_cell_size = [2, 3, 4]
for length in lengths:
    for size in unit_cell_size:
        sides = randomise_sides(length, 0)
        cubic = make_dataset(sides["a"], sides["b"], sides["c"], 90, 90, 90, "cubic")

        sides = randomise_sides(length, 0)
        hexagonal = make_dataset(sides["a"], sides["b"], sides["c"], 90, 90, 120, "hexagonal")

        sides = randomise_sides(length, 1)
        tetragonal = make_dataset(sides["a"], sides["b"], sides["c"], 90, 90, 90, "tetragonal")

        sides = randomise_sides(length, 2)
        orthorhombic = make_dataset(sides["a"], sides["b"], sides["c"], 90, 90, 90, "orthorhombic")

        sides = randomise_sides(length, 0)
        angles = randomise_angles(length, False, False, True)
        angles = pd.DataFrame(angles, columns=['alpha', 'beta', 'gamma'])
        rhomobohedral = make_dataset(sides["a"], sides["b"], sides["c"], angles["alpha"], 
                                    angles["beta"], angles["gamma"], "rhomobohedral")

        sides = randomise_sides(length, 2)
        angles = randomise_angles(length, True, False, False)
        monoclinc = make_dataset(sides["a"], sides["b"], sides["c"], 90, angles, 90, "monoclinc")

        sides = randomise_sides(length, 2)
        angles = randomise_angles(length, False, True, False)
        angles = pd.DataFrame(angles, columns=['alpha', 'beta', 'gamma'])

        triclinic = make_dataset(sides["a"], sides["b"], sides["c"],
                                    angles["alpha"], angles["beta"], angles["gamma"], "triclinic")

        final_df = pd.concat([cubic, hexagonal, rhomobohedral, tetragonal, orthorhombic, monoclinc, triclinic],ignore_index=True)

        coords = []
        for index, row in final_df.iterrows():
            coords.append(st.Structure(size,size,size,
                                    row["a"] ,row["b"], row["c"], 
                                    row["alpha"], row["beta"], row["gamma"],
                                    False, False, False))
            
        matrix_list, feat_cryst_list = featurising_coords(coords_of_structures=coords)
        final_df['Crystals Featurised'] = feat_cryst_list

        final_df["Lowest distortion"] = final_df["structure type"].astype('category')
        final_df["Lowest distortion Categories"] = final_df["Lowest distortion"].cat.codes
        
        print(len(final_df), size, length)
        f1, precision, recall, accuracy = do_random_forest(final_df["Crystals Featurised"], final_df["Lowest distortion Categories"])

700 2 100
Accuracy of the random forest: 25.71%
F1 Score: 0.11670197272606911
Precision: 0.24386128364389234
Recall: 0.2571428571428571
700 3 100
Accuracy of the random forest: 25.71%
F1 Score: 0.11904516417633615
Precision: 0.21049676351046215
Recall: 0.2571428571428571
700 4 100
Accuracy of the random forest: 25.71%
F1 Score: 0.11628919860627178
Precision: 0.22209728962007338
Recall: 0.2571428571428571
1400 2 200
Accuracy of the random forest: 28.21%
F1 Score: 0.134392698176276
Precision: 0.22769302615193027
Recall: 0.28214285714285714
1400 3 200
Accuracy of the random forest: 29.29%
F1 Score: 0.16156138205705475
Precision: 0.3440707539122478
Recall: 0.29285714285714287
1400 4 200
Accuracy of the random forest: 27.86%
F1 Score: 0.13252734955064766
Precision: 0.1907510876691102
Recall: 0.2785714285714286
2800 2 400
Accuracy of the random forest: 28.21%
F1 Score: 0.14335704983440403
Precision: 0.2958021677570549
Recall: 0.28214285714285714
2800 3 400
Accuracy of the random forest: 28.5